In [34]:
from collections import defaultdict
from dataclasses import dataclass, field
from enum import IntEnum
from typing import Mapping, Iterator
from typing import TypeVar, Set, Union, Optional

from more_itertools import powerset

In [35]:
@dataclass(frozen=True, order=True)
class Atom:
    symbol: str

    def __str__(self):
        return self.symbol

In [36]:
Alphabet = Set[Atom]
Valuation = Mapping[Atom, bool]

In [37]:
class Connective(IntEnum):
    And = 0
    Or = 1
    Implies = 2

    def __str__(self):
        if self is Connective.And:
            return "∧"
        elif self is Connective.Or:
            return "∨"
        elif self is Connective.Implies:
            return "→"
        else:
            assert False, "Unhandled Connective.__str__: {} = {}".format(self.name, self.value)

    def evaluate(self, left: bool, right: bool):
        if self is Connective.And:
            return left and right
        elif self is Connective.Or:
            return left or right
        elif self is Connective.Implies:
            return not left or right
        else:
            assert False, "Unhandled Connective.evaluate: {} = {}".format(self.name, self.value)


In [38]:
@dataclass(frozen=True, order=True)
class Literal:
    atom: Atom
    sign: bool = field(default=True)

    def __str__(self):
        sign_str = ""
        if not self.sign:
            sign_str = "¬"
        return "{}{}".format(sign_str, self.atom)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        return Literal(self.atom, not self.sign)

    def __and__(self, other):
        left = Formula(self)
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.And, right)

    def __or__(self, other):
        left = Formula(self)
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.Or, right)

    def __rshift__(self, other):
        left = Formula(self)
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.Implies, right)

    def __truediv__(self, other):
        left = Formula(self)
        right = other
        if isinstance(other, Literal):
            right = Formula(other)
        return Normally(left, right)

In [39]:
@dataclass(frozen=True, order=True)
class Top(Literal):
    atom: Atom = field(default=Atom('⊤'), init=False)
    sign: bool = field(default=True, init=False)

    def __str__(self):
        return str(self.atom)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        return Bot()

    def __and__(self, other):
        if isinstance(other, Literal):
            return Formula(other)
        return other

    def __or__(self, other):
        return Formula(Top())


In [40]:
@dataclass(frozen=True, order=True)
class Bot(Literal):
    atom: Atom = field(default=Atom('⊥'), init=False)
    sign: bool = field(default=False, init=False)

    def __neg__(self):
        return Top()

    def __str__(self):
        return str(self.atom)

    def __and__(self, other):
        return Formula(Bot())

    def __or__(self, other):
        if isinstance(other, Literal):
            return Formula(other)
        return other

In [41]:
ForwardFormula = TypeVar('ForwardFormula', bound='Formula')

@dataclass(frozen=True, order=True)
class Formula:
    left: Union[ForwardFormula, Literal]
    connective: Optional[Connective] = field(default=None)
    right: Union[ForwardFormula, None] = field(default=None)

    def __str__(self):
        left_str = str(self.left)
        connective_str = ""
        if self.connective is not None:
            connective_str = " {}".format(self.connective)
        right_str = ""
        if self.right is not None:
            if self.right.left == Bot() and self.right.right is None:
                left_str = "¬({})".format(left_str)
                connective_str = ""
            else:
                right_str = " {}".format(self.right)
        return "{}{}{}".format(left_str, connective_str, right_str)

    def __repr__(self):
        return str(self)

    def evaluate(self, valuation: Valuation) -> bool:
        if isinstance(self.left, Literal):
            value_left = self.__evaluate_literal(self.left, valuation)
        else:
            assert isinstance(self.left, Formula), "Unknown type for Formula.left. {}: {}".format(
                type(self.left).__name__, self.left)
            value_left = self.left.evaluate(valuation)
        if self.connective is not None and self.right is None:
            raise TypeError("Formula.connective present, despite Formula.right missing.")
        elif self.connective is None and self.right is not None:
            raise TypeError("Formula.connective missing, despite Formula.right present.")

        if self.connective is None and self.right is None:
            return value_left
        else:
            assert isinstance(self.right, Formula), "Unknown type for Formula.right. {}: {}".format(
                type(self.right).__name__, self.right)
            value_right = self.right.evaluate(valuation)

            return self.connective.evaluate(value_left, value_right)

    def __evaluate_literal(self, literal: Literal, valuation: Valuation) -> bool:
        if isinstance(literal, Top) or isinstance(literal, Bot):
            return literal.sign
        else:
            # get assigned truth value of atom (per default false) and flip the result if negated
            return bool(valuation.get(literal.atom, False) ^ (not literal.sign))

    @property
    def literals(self) -> Set[Literal]:
        literals = set()
        if isinstance(self.left, Literal):
            if not isinstance(self.left, Top) and not isinstance(self.left, Bot):
                literals.add(self.left)
        else:
            assert isinstance(self.left, Formula), "Unknown type for Formula.right. {}: {}".format(
                type(self.left).__name__, self.left)
            literals.update(self.left.literals)
        if self.right is not None:
            assert isinstance(self.right, Formula), "Unknown type for Formula.right. {}: {}".format(
                type(self.right).__name__, self.right)
            literals.update(self.right.literals)
        return literals

    @property
    def atoms(self) -> Set[Atom]:
        return {literal.atom for literal in self.literals}

    def __neg__(self):
        if self.connective is not None and self.right is None:
            raise TypeError("Formula.connective present, despite Formula.right missing.")
        elif self.connective is None and self.right is not None:
            raise TypeError("Formula.connective missing, despite Formula.right present.")

        if self.connective is None and self.right is None:
            return Formula(-self.left)
        elif self.connective is Connective.And:
            return Formula(-self.left, Connective.Or, -self.right)
        elif self.connective is Connective.Or:
            return Formula(-self.left, Connective.And, -self.right)
        elif self.connective is Connective.Implies:
            if self.right.left == Bot() and self.right.right is None:
                return self.left
            return Formula(self, Connective.Implies, Formula(Bot()))
        else:
            assert False, "Unknown Formula.connective. {} = {}.".format(self.connective.name, self.connective.value)

    def __and__(self, other):
        left = self
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.And, right)

    def __or__(self, other):
        left = self
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.Or, right)

    def __rshift__(self, other):
        left = self
        right = other
        if isinstance(other, Literal):
            right = Formula(right)
        return Formula(left, Connective.Implies, right)

    def __truediv__(self, other):
        left = self
        right = other
        if isinstance(other, Literal):
            right = Formula(other)
        return Normally(left, right)

    def __call__(self, valuation: Valuation) -> bool:
        return self.evaluate(valuation)


In [42]:
@dataclass(frozen=True, order=True)
class Normally:
    left: Formula
    right: Formula

    def __str__(self):
        if self.right.left == Bot() and self.right.right is None and self.left.right.left == Bot() and self.left.right.right is None:
            return str(self.left.left)
        return "{} |~ {}".format(self.left, self.right)

    def __repr__(self):
        return str(self)

    @property
    def literals(self) -> Set[Literal]:
        return {literal for formula in (self.left, self.right) for literal in formula.literals}

    @property
    def atoms(self) -> Set[Atom]:
        return {atom for formula in (self.left, self.right) for atom in formula.atoms}

    @property
    def is_classical(self) -> bool:
        return self.right.left == Bot() and self.right.right is None

    def materialize(self):
        if self.right.left == Bot() and self.right.right is None:
            if self.left.right.left == Bot() and self.left.right.right is None:
                return self.left.left
            return -self.left
        return self.left >> self.right


In [43]:
KnowledgeBase = Set[Normally]

In [44]:
def print_knowledge_base(knowledge_base: KnowledgeBase):
    print("{", ', '.join(map(str, knowledge_base)), "}")

In [45]:
def all_valuations(alphabet: Alphabet, complete: bool = False) -> Iterator[Valuation]:
    subsets = powerset(alphabet)
    for subset in subsets:
        valuation = defaultdict(lambda: False)
        for atom in subset:
            valuation[atom] = True
        if complete:
            for atom in alphabet:
                if atom not in subset:
                    valuation[atom] = False
        yield valuation


In [46]:
def print_valuation(valuation: Valuation, alphabet: Optional[Alphabet] = None):
    trues = set()
    falses = set()
    for atom, value in valuation.items():
        if value:
            trues.add(atom)
        else:
            falses.add(atom)
    if alphabet is not None:
        falses.update(alphabet - trues)
        alph_str_len = len(''.join(map(str, alphabet)))
        pad = alph_str_len + len(alphabet) - 1
    else:
        alph_str_len = len(''.join(map(str, trues))) + len(''.join(map(str, falses)))
        pad = alph_str_len + len(trues) + len(falses)

    print("{", ' '.join(map(str, trues)).ljust(pad), "}", "{", ' '.join(map(str, falses)).ljust(pad), "}")


In [47]:
def models(formulas: Set[Formula], alphabet: Optional[Alphabet] = None) -> Iterator[Valuation]:
    if alphabet is None:
        alphabet = {atom for formula in formulas for atom in formula.atoms}
    for valuation in all_valuations(alphabet):
        if all(formula.evaluate(valuation) for formula in formulas):
            yield valuation


In [48]:
def sat(formulas: Set[Formula], alphabet: Optional[Alphabet] = None) -> bool:
    model = next(models(formulas, alphabet), None)
    return model is not None

In [49]:
def unsat(formulas: Set[Formula], alphabet: Optional[Alphabet] = None) -> bool:
    return not sat(formulas, alphabet)

In [50]:
def entails(formulas: Set[Formula], formula: Formula) -> bool:
    return unsat(formulas | {-formula})

In [51]:
def valid(formulas: Set[Formula], alphabet: Optional[Alphabet] = None) -> bool:
    if alphabet is None:
        alphabet = {atom for formula in formulas for atom in formula.atoms}
    for valuation in all_valuations(alphabet):
        if any(not formula.evaluate(valuation) for formula in formulas):
            return False
    return True

In [52]:
def materialized(knowledge_base: KnowledgeBase) -> Set[Formula]:
    return {statement.materialize() for statement in knowledge_base}


In [53]:
f = Literal(Atom('f')) # flies
b = Literal(Atom('b')) # is a bird
p = Literal(Atom('p')) # is a pengiun
w = Literal(Atom('w')) # has wings
bot = Bot()            # Falsum
top = Top()            # Verum

In [54]:
f # it flies

f

In [55]:
-f # it does not fly

¬f

In [56]:
b >> f # birds fly

b → f

In [57]:
p >> -f # pengiuns don't fly

p → ¬f

In [58]:
p & b >> w # given that it is a penguin and a bird, it has wings

p ∧ b → w

In [59]:
(p & b >> w)({p.atom: True, b.atom: True, w.atom: True}) # evaluate where pbw are true

True

In [60]:
(p & b >> w)({p.atom: True, b.atom: True, w.atom: False}) # evaluate where pb are true and w is false

False

In [61]:
p / -f # penguins normally don't fly

p |~ ¬f

In [62]:
K = {((p >> b) >> Bot()) / Bot(), b / f, p / -f, b / w}

In [63]:
print_knowledge_base(K)

{ p |~ ¬f, p → b, b |~ w, b |~ f }


In [64]:
alph = {lit.atom for lit in (b, p, f)}
tuple(print_valuation(valuation, alph) for valuation in all_valuations(alph));

{       } { b p f }
{ b     } { p f   }
{ f     } { b p   }
{ p     } { b f   }
{ b f   } { p     }
{ b p   } { f     }
{ p f   } { b     }
{ b p f } {       }


In [65]:
tuple(print_valuation(model, alph) for model in models({p >> b, b >> f, p >> -f}));

{       } { b p f }
{ f     } { b p   }
{ b f   } { p     }


In [66]:
materialized(K)

{b → f, b → w, p → b, p → ¬f}